In [1]:
import pandas as pd
import os

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
study_name = COMBINE_harmonizer.STUDY_OC
sheet_name = COMBINE_harmonizer.SHEET_FOLLOW_UP

root_dir = '..'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name)

In [6]:
dict_columns = list(df_dict[study_name].unique())

## 03. OC

In [7]:
the_dir = cfg.config[f'{study_name}_follow_up_dir']

In [8]:
filenames = os.listdir(the_dir)
filenames.sort()

full_filenames = [os.sep.join([the_dir, each_filename]) for each_filename in filenames]

In [9]:
analysis_dir = cfg.config[f'{study_name}_analysis_dir']

In [10]:
analysis_filenames = os.listdir(analysis_dir)
analysis_filenames.sort()

full_analysis_filenames = [os.sep.join([analysis_dir, each_filename]) for each_filename in analysis_filenames]

In [11]:
all_filenames = filenames + analysis_filenames
full_all_filenames = full_filenames + full_analysis_filenames
dfs = [pd.read_csv(each_filename) for each_filename in full_all_filenames]

### 03.1. check columns in each file

In [12]:
for idx, each_df in enumerate(dfs):
    print(f'({idx}/{len(dfs)}) filename: {all_filenames[idx]} columns: {list(each_df.columns)}')

(0/12) filename: of01.csv columns: ['LHFOLNUM', 'SITENM', 'LAST', 'FIRST', 'HTHRM_ID', 'PROTID02', 'PROTID03', 'PROTID04', 'PROTID05', 'PROTID06', 'PROTID07', 'PROTID08', 'PROTID09', 'PROTID10', 'PROTID11', 'PROTID12', 'PROTID13', 'PROTID14', 'PROTID15', 'PROTID16', 'PROTID17', 'PROTID18', 'PROTID19', 'PROTID20', 'REC_CMP', 'CENTER', 'BIRTHDT', 'VISITDT', 'BIRTHNM', 'PREVCNTR', 'CMP_DATE', 'CRT_DATE']
(1/12) filename: of03.csv columns: ['LHFOLNUM', 'CFSUPER', 'NF3MONEY', 'CFWKPCAR', 'CFWKOCAR', 'CFSCCARE', 'CFSCADUL', 'CFOTHLAN', 'CFSECLAN', 'CFOTHSEC', 'CFNDNRSV', 'CFNDNRSH', 'CFNDOTPT', 'CFNDSPCH', 'CFNDEALY', 'CFNDSOCL', 'CFNDPUL', 'CFNDOPH', 'CFNDGAS', 'CFNDAUD', 'CFNDNEU', 'CFNDOTH', 'CFCVOSP', 'CFNDEVAL', 'CFREGDR', 'CFCCFAC', 'CFCAROT', 'CFTRDYCR', 'CFSPDYCR', 'CFHMDYCR', 'CFBBYSIT', 'CFINITS', 'REC_CMP', 'CENTER', 'CFVISDT', 'CFBRTHDT', 'CFCHRAGE', 'CFADJAGE', 'CFCARE', 'CFOCAR', 'CFMARITL', 'CFLIVING', 'CFPEOPLE', 'CFGRPCAR', 'CFGROCAR', 'CFINSUR', 'CFLANG', 'CFSECOND', 'CFNMP

### 03.2 all columns

In [13]:
all_columns = []
for idx, each_df in enumerate(dfs):
    all_columns += list(map(lambda x: {'column': x, 'idx': idx, 'filename': all_filenames[idx]}, list(each_df.columns)))
df_columns = pd.DataFrame(all_columns)
df_columns['column:filename'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}", axis=1)
df_columns['column:filename:idx'] = df_columns.apply(lambda x: f"{x['column']}:{x['filename']}:{x['idx']}", axis=1)
df_columns.sort_values(by=['filename', 'column'], inplace=True)

In [14]:
df_columns

,column,idx,filename,column:filename,column:filename:idx
438,CENTER,10,analysis.csv,CENTER:analysis.csv,CENTER:analysis.csv:10
437,HTHRM_ID,10,analysis.csv,HTHRM_ID:analysis.csv,HTHRM_ID:analysis.csv:10
529,MRI_2cat,10,analysis.csv,MRI_2cat:analysis.csv,MRI_2cat:analysis.csv:10
531,acidosis,10,analysis.csv,acidosis:analysis.csv,acidosis:analysis.csv:10
457,adjudicate,10,analysis.csv,adjudicate:analysis.csv,adjudicate:analysis.csv:10
...,...,...,...,...,...
401,NF12NRDS,9,of12.csv,NF12NRDS:of12.csv,NF12NRDS:of12.csv:9
400,NF12NRPB,9,of12.csv,NF12NRPB:of12.csv,NF12NRPB:of12.csv:9
434,OF12CHGM,9,of12.csv,OF12CHGM:of12.csv,OF12CHGM:of12.csv:9
420,OF12GMFL,9,of12.csv,OF12GMFL:of12.csv,OF12GMFL:of12.csv:9


In [15]:
df_columns_count = df_columns.groupby(['column']).agg(_count=('column', 'count')).sort_values(by=['_count'], ascending=False)
df_columns_count.head(20)

,_count
column,
CENTER,12
LHFOLNUM,10
REC_CMP,10
CRT_DATE,10
CMP_DATE,10
HTHRM_ID,3
vent_max,2
edema,2
death18,2


### 03.3 check LHFOLNUM

In [16]:
is_LM3ID = df_columns['column'] == 'LHFOLNUM'
df_columns[is_LM3ID]

,column,idx,filename,column:filename,column:filename:idx
0,LHFOLNUM,0,of01.csv,LHFOLNUM:of01.csv,LHFOLNUM:of01.csv:0
32,LHFOLNUM,1,of03.csv,LHFOLNUM:of03.csv,LHFOLNUM:of03.csv:1
106,LHFOLNUM,2,of04.csv,LHFOLNUM:of04.csv,LHFOLNUM:of04.csv:2
178,LHFOLNUM,3,of04a.csv,LHFOLNUM:of04a.csv,LHFOLNUM:of04a.csv:3
188,LHFOLNUM,4,of04ar.csv,LHFOLNUM:of04ar.csv,LHFOLNUM:of04ar.csv:4
199,LHFOLNUM,5,of05.csv,LHFOLNUM:of05.csv,LHFOLNUM:of05.csv:5
296,LHFOLNUM,6,of09a.csv,LHFOLNUM:of09a.csv,LHFOLNUM:of09a.csv:6
342,LHFOLNUM,7,of10.csv,LHFOLNUM:of10.csv,LHFOLNUM:of10.csv:7
356,LHFOLNUM,8,of11.csv,LHFOLNUM:of11.csv,LHFOLNUM:of11.csv:8
373,LHFOLNUM,9,of12.csv,LHFOLNUM:of12.csv,LHFOLNUM:of12.csv:9


##### column not in dict

In [17]:
is_not_in_dict = df_columns['column'].isin(dict_columns) == False
df_columns_not_in_dict = df_columns[is_not_in_dict]
df_columns_not_in_dict

,column,idx,filename,column:filename,column:filename:idx
529,MRI_2cat,10,analysis.csv,MRI_2cat:analysis.csv,MRI_2cat:analysis.csv:10
531,acidosis,10,analysis.csv,acidosis:analysis.csv,acidosis:analysis.csv:10
458,age_death,10,analysis.csv,age_death:analysis.csv,age_death:analysis.csv:10
468,age_rand,10,analysis.csv,age_rand:analysis.csv,age_rand:analysis.csv:10
524,all_cere,10,analysis.csv,all_cere:analysis.csv,all_cere:analysis.csv:10
...,...,...,...,...,...
435,CMP_DATE,9,of12.csv,CMP_DATE:of12.csv,CMP_DATE:of12.csv:9
436,CRT_DATE,9,of12.csv,CRT_DATE:of12.csv,CRT_DATE:of12.csv:9
402,LFINITA,9,of12.csv,LFINITA:of12.csv,LFINITA:of12.csv:9
406,NF12INIT,9,of12.csv,NF12INIT:of12.csv,NF12INIT:of12.csv:9


In [18]:
columns_not_in_dict = list(df_columns_not_in_dict['column:filename:idx'].unique())
# columns_not_in_dict.sort()
columns_not_in_dict

['MRI_2cat:analysis.csv:10',
 'acidosis:analysis.csv:10',
 'age_death:analysis.csv:10',
 'age_rand:analysis.csv:10',
 'all_cere:analysis.csv:10',
 'anticon_0:analysis.csv:10',
 'any_antic:analysis.csv:10',
 'any_inot:analysis.csv:10',
 'any_pse:analysis.csv:10',
 'any_seiz:analysis.csv:10',
 'apg10_lt5:analysis.csv:10',
 'apgar10_5:analysis.csv:10',
 'apgar5_5:analysis.csv:10',
 'bel_max:analysis.csv:10',
 'bg_max:analysis.csv:10',
 'bs_max:analysis.csv:10',
 'cc_max:analysis.csv:10',
 'cereb_any:analysis.csv:10',
 'cr_max:analysis.csv:10',
 'dual_base:analysis.csv:10',
 'dual_ph:analysis.csv:10',
 'ecsect:analysis.csv:10',
 'edema:analysis.csv:10',
 'ex_max:analysis.csv:10',
 'extent_num:analysis.csv:10',
 'fcenter:analysis.csv:10',
 'fp_max:analysis.csv:10',
 'frontal:analysis.csv:10',
 'hemi_lat:analysis.csv:10',
 'hie_fix:analysis.csv:10',
 'hip_max:analysis.csv:10',
 'hy_max:analysis.csv:10',
 'inot_0:analysis.csv:10',
 'ins_max:analysis.csv:10',
 'lateral_num:analysis.csv:10',
 '

#### dict not in column

In [19]:
is_dict_not_in_columns = (df_dict['OC'].isin(df_columns['column']) == False) & (df_dict['OC'].isnull() == False)
df_dict_not_in_columns = df_dict[is_dict_not_in_columns]
df_dict_not_in_columns

,Category,Subcategory,Standardized_VariableNames_Dictionary,type,Variable_Description,#studies w/ this var,redcap,comment,lower_var,var_eq_redcap,connect_redcap,LH,OC


## 04. show column values

In [20]:
# XXX no show due to data privacy concern.

'''
for idx, each_df in enumerate(dfs):
    for idx2, column in enumerate(each_df.columns):
        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')
'''

"\nfor idx, each_df in enumerate(dfs):\n    for idx2, column in enumerate(each_df.columns):\n        print(f'({idx}/{len(dfs)}/{filenames[idx]}) ({idx2}/{len(each_df.columns)} column: {column} value: {list(each_df[column].unique())}')\n"